# ИМПОРТЫ

In [5]:
%%capture
!pip install -q --upgrade --no-cache-dir --no-deps unsloth, transformers accelerate peft

# ОБУЧЕНИЕ БАЗОВОЙ МОДЕЛИ

## Импортируем модель и вводим параметры

In [6]:
# import unsloth
from unsloth import FastLanguageModel
import torch


max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

output_dir = 'Qwen2-1.5B-bnb-4bit-kl-sft-base1' # Введите свое название модели после файнтюнинга
model_name = "unsloth/Qwen2-1.5B-bnb-4bit" # Введите название модели
# model_name = 'unsloth/gemma-2-2b-bnb-4bit'
# model_name = 'unsloth/mistral-7b-v0.3-bnb-4bit'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'bitsandbytes'

## Добавляем параметры LoRa

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.9.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Подготавливаем датасет

In [4]:
import pandas as pd
train_data = pd.read_excel('/kaggle/input/dataf0/data 0.xlsx')

print(train_data[['description', 'target']].head())

y = train_data['target']
X_train = train_data[['description', 'target']]

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)


alpaca_prompt = """
### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["description"]
    outputs = examples["target"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(X_train),
})

train_dataset_dict = train_dataset_dict.map(formatting_prompts_func, batched = True)
train_dataset_dict

                                         description  \
0  л14 материал представлен содержимым кистозно-г...   
1  П 29 (№3683/20). На фоне "жидкого" коллоидаизм...   
2  л22 в мазке  неравномерной толщины с обильным ...   
3  пунктирован узел коллоидного в разной степени ...   
4  П6 в мазке высокой клеточности фоне кистозно г...   

                                              target  
0  В соответствии с критериями   системы классифи...  
1  Неинформативное исследование в соответствии с ...  
2  В соответствии с критериями системы классифика...  
3                            Bethesda - категория II  
4                             Bethesda - категория V  


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['description', 'target', 'text'],
        num_rows: 1000
    })
})

In [5]:
train_dataset_dict['train'][0]

{'description': 'л14 материал представлен содержимым кистозно-геморрагической полостиобильным плотным  коллоидом  на фоне которых обнаружены фрагменты лизированных клеток и  единичные мелкие бесформенные скопления  полиморфных   эпителиальных клеток с выраженными дегенеративными изменениями.',
 'target': 'В соответствии с критериями   системы классификации Bethesdа: материанедостаточно информативный диагностическая категория I',
 'text': '\n### Input:\nл14 материал представлен содержимым кистозно-геморрагической полостиобильным плотным  коллоидом  на фоне которых обнаружены фрагменты лизированных клеток и  единичные мелкие бесформенные скопления  полиморфных   эпителиальных клеток с выраженными дегенеративными изменениями.\n\n### Output:\nВ соответствии с критериями   системы классификации Bethesdа: материанедостаточно информативный диагностическая категория I<|endoftext|>'}

## Вводим параметры обучения

In [6]:
import torch
from trl import SFTTrainer
from transformers.trainer import Accelerator
import warnings

class CustomSFTTrainer_w_kl(SFTTrainer):
    def __init__(self, kl_alpha=0.1, **kwargs):
        
        super().__init__(**kwargs)

        if not hasattr(model, 'disable_adapter'):
            warnings.warn(
                "The model provided does not appear to be a PEFT model with a LoRA adapter. "
                "The KL divergence loss term will not be calculated and the trainer will fall back to standard SFT."
            )

        self.accelerator = Accelerator()
        self._total_train_tokens = 0
        # After super().__init__, self._metrics is initialized by the parent Trainer class.
        # We need to add our custom metric to it.
        if "kl_loss" not in self._metrics["train"]:
            self._metrics["train"]["kl_loss"] = []
        if "eval" in self._metrics and "kl_loss" not in self._metrics["eval"]:
            self._metrics["eval"]["kl_loss"] = []
        self.kl_alpha = kl_alpha


    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Computes SFT loss and adds a KL divergence penalty term.
        The reference log probabilities are computed on the fly by disabling the LoRA adapter.
        """
        if self.model_accepts_loss_kwargs:
            loss_kwargs = {}
            if num_items_in_batch is not None:
                loss_kwargs["num_items_in_batch"] = num_items_in_batch
        # Compute standard SFT loss with the adapter enabled
        sft_loss, outputs = super().compute_loss(model, inputs, return_outputs=True, **loss_kwargs)

        # If the model is not a PEFT model, we can't compute KL loss, so we return SFT loss
        if not hasattr(model, 'disable_adapter'):
            warnings.warn(
                "The model provided does not appear to be a PEFT model with a LoRA adapter. "
                "The KL divergence loss term will not be calculated and the trainer will fall back to standard SFT."
            )
            return (sft_loss, outputs) if return_outputs else sft_loss

        # Get the logits from the adapter-enabled model
        logits = outputs.logits
        labels = inputs["labels"]

        # Compute reference logits by disabling the adapter
        with torch.no_grad(), model.disable_adapter():
            ref_outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask", None),
            )
        ref_logits = ref_outputs.logits
        
        # print(ref_logits)
        # Shift all tensors for next-token prediction
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_ref_logits = ref_logits[..., :-1, :].contiguous()

        # Create a mask for the answer tokens (non -100 labels)
        answer_mask = (shift_labels != -100).float()

        # Calculate log probabilities from the adapter model and reference model logits
        log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
        ref_log_probs = torch.nn.functional.log_softmax(shift_ref_logits, dim=-1)

        # Calculate KL divergence: KL(P_ref || P_model) = sum(P_ref * (log P_ref - log P_model))
        kl_divergence = torch.exp(ref_log_probs) * (ref_log_probs - log_probs)

        # Sum over the vocabulary dimension and apply the mask and average over the sequence and batch
        kl_loss_per_token = (kl_divergence.sum(dim=-1) * answer_mask)
        kl_loss = kl_loss_per_token.sum() / answer_mask.sum()

        # Combine losses
        loss = sft_loss + self.kl_alpha * kl_loss

        # Logging
        self._metrics["train"]["kl_loss"].append(kl_loss.item())

        return (loss, outputs) if return_outputs else loss

from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = False,
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

# Инициализация и запуск
trainer = CustomSFTTrainer_w_kl(
    model=model,  # PeFT модель с LoRA
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    tokenizer=tokenizer,
    kl_alpha=0.05
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

## TEMA тренер

In [8]:
import copy
import torch
import torch.nn.functional as F
from trl import SFTTrainer
from peft import PeftModel

class EMATeacherGPUTrainer(SFTTrainer):
    """
    EMA teacher trainer (GPU-only).
    - teacher is a deepcopy of the model, placed on the same device and cast to float32.
    - EMA updates only floating parameters by name (teacher_param = decay * teacher_param + (1-decay) * student_param).
    - Designed for non-sharded single-GPU (or standard DataParallel) runs.
    WARNING: doubles GPU memory usage.
    """
    def __init__(self, ema_decay=0.999, alpha=1.0, beta=0.05, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.ema_decay = float(ema_decay)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.temperature = float(temperature)
        self.teacher = None
        self._ema_initialized = False
        self._student_param_map = None  # name -> param (view into student model)

    def create_teacher(self):
        if self.teacher is not None:
            return
        model_device = next(self.model.parameters()).device
        # deepcopy ломает PeftModel, поэтому лучше:
        self.teacher = PeftModel.from_pretrained(self.model.base_model, self.model.peft_config).to(model_device)
    
        self.teacher.load_state_dict(self.model.state_dict(), strict=False)
        for p in self.teacher.parameters():
            p.data = p.data.to(torch.float32)
            p.requires_grad_(False)
        self.teacher.eval()
        self._student_param_map = dict(self.model.named_parameters())
        self._ema_initialized = True


    def _move_inputs_to_device(self, inputs, device):
        out = {}
        for k, v in inputs.items():
            if torch.is_tensor(v):
                out[k] = v.to(device)
            else:
                out[k] = v
        return out

    @torch.no_grad()
    def _update_ema(self):
        """
        Update teacher parameters with EMA from student.
        Only updates named parameters that exist in both teacher and student.
        Teacher params are assumed float32 and on the same device.
        """
        decay = float(self.ema_decay)
        if self.teacher is None or self._student_param_map is None:
            return

        # Iterate teacher named parameters and update from student param with same name
        for t_name, t_param in self.teacher.named_parameters():
            s_param = self._student_param_map.get(t_name, None)
            if s_param is None:
                # unexpected mismatch; skip
                continue

            # only update floating point parameters
            if not torch.is_floating_point(t_param):
                continue

            # read student param as float32 on teacher device
            s_data = s_param.detach().to(dtype=t_param.dtype, device=t_param.device)
            t_param.data.mul_(decay).add_(s_data, alpha=(1-decay))

            # EMA: t = decay * t + (1 - decay) * s
            # t_param is float32; do in-place ops on t_param.data

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
      # Ensure teacher initialized
      if not self._ema_initialized:
          self.create_teacher()

      model_device = next(model.parameters()).device
      # Prepare inputs for student (do not mutate original)
      inputs_for_student = {k: (v.to(model_device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
      if "labels" not in inputs_for_student:
          raise RuntimeError("No 'labels' in inputs_for_student — check data collator/tokenization")
      labels = inputs_for_student.pop("labels")  # on model_device

      # Student forward
      outputs = model(**inputs_for_student, labels=labels)
      ce_loss = outputs.loss
      logits = outputs.logits  # [B,S,V] on model_device

      # Teacher forward (teacher is on same device in EMATeacherGPUTrainer)
      teacher_device = next(self.teacher.parameters()).device
      teacher_inputs = {k: (v.to(teacher_device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
      teacher_inputs["use_cache"] = False
      with torch.no_grad():
          teacher_logits = self.teacher(**teacher_inputs).logits  # [B,S,V] on teacher_device

      # --- Diagnostics: check finite-ness ---
      if not torch.isfinite(teacher_logits).all():
        # print("[WARN] teacher_logits contains non-finite values (NaN/Inf). Replacing with zeros for stability.")

        # безопасно получаем min/max, игнорируя NaN
        finite_mask = torch.isfinite(teacher_logits)
        if finite_mask.any():
            t_min = teacher_logits[finite_mask].min().item()
            t_max = teacher_logits[finite_mask].max().item()
        else:
            t_min, t_max = float('nan'), float('nan')

        # print(f" teacher_logits stats (finite only): min={t_min:.6g}, max={t_max:.6g}")

        teacher_logits = torch.nan_to_num(
            teacher_logits,
            nan=0.0,
            posinf=1e4,
            neginf=-1e4
        )

      # Masking valid tokens
      valid_mask = labels.ne(-100)  # [B, S]
      num_valid = int(valid_mask.sum().item())
      if num_valid == 0:
          raise ValueError("No valid tokens in labels (all -100). Stop training and fix data.")

      # Select valid positions and cast to float32 for KL computation
      student_logits_sel = logits[valid_mask].float()         # [N_valid, V] on model_device
      teacher_logits_sel = teacher_logits.to(logits.device)[valid_mask].float()  # ensure same device

      T = float(self.temperature)
      V = student_logits_sel.size(-1)

      # Stable teacher log-probs and probs
      teacher_logp = F.log_softmax(teacher_logits_sel / T, dim=-1)  # may contain large negative numbers
      # compute teacher_p as exp(teacher_logp) in float32
      teacher_p = torch.exp(teacher_logp)

      # Option 1: clamp teacher_p to avoid exact zeros (recommended)
      eps = 1e-12
      teacher_p = teacher_p.clamp(min=eps)
      # recompute teacher_logp from clamped teacher_p to keep consistency
      teacher_logp = torch.log(teacher_p)

      # Optionally apply smoothing to teacher distribution (helps if teacher is extremely peaky)
      smoothing = 0.0  # set small value like 1e-6..1e-4 to try; here default 0
      if smoothing > 0.0:
          # teacher_p <- (1 - s) * teacher_p + s / V
          teacher_p = teacher_p * (1.0 - smoothing) + (smoothing / float(V))
          teacher_logp = torch.log(teacher_p.clamp(min=eps))

      # student_logp (already stable because log_softmax)
      student_logp = F.log_softmax(student_logits_sel / T, dim=-1)

      # Compute per-element KL contribution: teacher_p * (teacher_logp - student_logp)
      # This is numerically more explicit than relying on F.kl_div under extreme tails.
      per_token_per_vocab = teacher_p * (teacher_logp - student_logp)  # shape [N_valid, V]

      # Replace NaN/Infs in contributions just in case (shouldn't happen after clamp, but safe)
      per_token_per_vocab = torch.nan_to_num(per_token_per_vocab, nan=0.0, posinf=0.0, neginf=0.0)

      # Sum over vocab then over tokens
      kl_sum = per_token_per_vocab.sum()  # scalar
      kl_sum = kl_sum * (T ** 2)

      kl_loss = kl_sum / float(num_valid)

      # If kl_loss is still nan (very unlikely), fallback to small value and warn
      if not torch.isfinite(kl_loss):
          print("[WARN] kl_loss is non-finite after stabilization; replacing with 0.0")
          kl_loss = torch.tensor(0.0, device=ce_loss.device, dtype=ce_loss.dtype)

      total = self.alpha * ce_loss + self.beta * kl_loss

      # Debug prints
      ce_f = float(ce_loss.detach().cpu().item())
      kl_f = float(kl_loss.detach().cpu().item())
      total_f = float(total.detach().cpu().item())
      # print(f"[TRAIN-STEP DEBUG] ce_loss={ce_f:.6g}, kl_loss={kl_f:.6g}, total={total_f:.6g}, num_valid={num_valid}")

      # EMA update
      try:
          with torch.no_grad():
              self._update_ema()
      except Exception as e:
          print(f"[WARN] EMA update failed: {e}")

      # Log with trainer
      try:
          self.log({"train/ce_loss": ce_f, "train/kl_loss": kl_f, "train/total_loss": total_f, "train/num_valid": num_valid})
      except Exception:
          pass

      return (total, outputs) if return_outputs else total


from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = False,
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

# Инициализация и запуск
trainer = EMATeacherGPUTrainer(
    model=model,  # PeFT модель с LoRA
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    tokenizer=tokenizer,
    ema_decay=0.999,
    alpha=1.0, beta=0.05, temperature=1.0
)


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [7]:
import torch
import torch.nn.functional as F
from trl import SFTTrainer
from peft import PeftModel

class LoRAEMATrainer(SFTTrainer):
    """
    EMA teacher trainer (GPU-only, optimized for LoRA).
    - EMA обновляет только веса LoRA (A, B матрицы).
    - Teacher не хранит всю модель -> только EMA-копию LoRA-параметров.
    - Это сильно экономит память: нет второй копии всей LLM.
    """
    def __init__(self, ema_decay=0.999, alpha=1.0, beta=0.05, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.ema_decay = float(ema_decay)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.temperature = float(temperature)

        # EMA-хранилище только для LoRA-параметров
        self.ema_state = {}
        self._init_ema()

    def _init_ema(self):
        """Сохраняем только LoRA-параметры (только trainable)."""
        for name, param in self.model.named_parameters():
            if param.requires_grad:  # только LoRA
                self.ema_state[name] = param.detach().clone().float()

    @torch.no_grad()
    def _update_ema(self):
        """EMA-обновление только LoRA параметров."""
        decay = self.ema_decay
        for name, param in self.model.named_parameters():
            if name in self.ema_state:
                ema_param = self.ema_state[name]
                # обновляем inplace
                ema_param.mul_(decay).add_(param.detach().float(), alpha=(1.0 - decay))

    def _apply_ema_weights(self):
        """Создаём копию LoRA state_dict с EMA-весами."""
        ema_lora_state = {}
        for name, param in self.model.named_parameters():
            if name in self.ema_state:
                ema_lora_state[name] = self.ema_state[name].clone().to(param.device)
        return ema_lora_state

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # ===== Student pass =====
        labels = inputs.pop("labels")
        outputs = model(**inputs, labels=labels)
        ce_loss = outputs.loss
        student_logits = outputs.logits

        
        # ===== Teacher pass (на EMA-весах) =====
        with torch.no_grad():
            ema_state = self._apply_ema_weights()

            # временно подменяем веса модели
            backup = {}
            for name, param in model.named_parameters():
                if name in ema_state:
                    backup[name] = param.data.clone()
                    param.data.copy_(ema_state[name].to(param.device))

            teacher_logits = model(**inputs, use_cache=False).logits


        
            # откатываем LoRA на студентские веса
            for name, param in model.named_parameters():
                if name in backup:
                    param.data.copy_(backup[name])

        # ===== KL Loss =====
        valid_mask = labels.ne(-100)
        student_sel = student_logits[valid_mask].float()
        teacher_sel = teacher_logits[valid_mask].float()

        T = self.temperature
        teacher_logp = F.log_softmax(teacher_sel / T, dim=-1)
        student_logp = F.log_softmax(student_sel / T, dim=-1)

        kl = F.kl_div(student_logp, teacher_logp.exp(), reduction="batchmean") * (T ** 2)

        # total loss
        total_loss = self.alpha * ce_loss + self.beta * kl

        # EMA update
        self._update_ema()

        return (total_loss, outputs) if return_outputs else total_loss

from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = False,
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

# Инициализация и запуск
trainer = LoRAEMATrainer(
    model=model,  # PeFT модель с LoRA
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    tokenizer=tokenizer,
    ema_decay=0.999,
    alpha=1.0, beta=0.05, temperature=1.0
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

## FT тренер

In [6]:
from trl import SFTTrainer
import torch
import torch.nn as nn


class CombinedLossSFTTrainer(SFTTrainer):
    def __init__(self, alpha=0.7, beta=0.3, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha
        self.beta = beta
        self.temperature = temperature
        # Сохраняем исходные веса модели для teacher
        self.original_state = {k: v.detach().clone() for k, v in self.model.state_dict().items()}
        self.teacher = None

    def create_teacher(self):
        # Создаём единожды глубокую копию текущей модели как teacher
        if self.teacher is None:
            import copy
            self.teacher = copy.deepcopy(self.model)
            self.teacher.to(self.model.device)
            self.teacher.eval()
            for p in self.teacher.parameters():
                p.requires_grad = False

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Обеспечиваем teacher
        self.create_teacher()
        labels = inputs.pop('labels')
        # Forward студента
        outputs = model(**inputs, labels=labels)
        ce_loss = outputs.loss
        logits = outputs.logits
        # Forward teacher без градиентов
        with torch.no_grad():
            teacher_logits = self.teacher(**inputs).logits
        # KL-дивергенция
        student_lp = nn.functional.log_softmax(logits / self.temperature, dim=-1)
        teacher_p = nn.functional.softmax(teacher_logits / self.temperature, dim=-1)
        kl_loss = nn.functional.kl_div(
            student_lp, teacher_p, reduction='batchmean'
        ) * (self.temperature ** 2)
        total = self.alpha * ce_loss + self.beta * kl_loss
        return (total, outputs) if return_outputs else total

from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

alpha = 1.0
beta = 0.05
temperature = 1.0

# Инициализация и запуск
trainer = CombinedLossSFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    # eval_dataset=eval_dataset,
    # data_collator=default_data_collator,
    tokenizer=tokenizer,
    alpha=alpha,
    beta=beta,
    temperature=temperature
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

## Обычный тренер

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset_dict['train'],
    dataset_text_field = "text",
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 4 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 9,232,384 of 1,552,946,688 (0.59% trained)


Step,Training Loss
10,3.066700
20,2.084700
30,1.497600
40,1.224500
50,1.143600
60,1.087600
70,1.057700
80,0.867000
90,0.959800
100,0.830900


Unsloth: Will smartly offload gradients to save VRAM!


## Сохраняем модель

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")

In [9]:
!huggingface-cli login --token $secret_value_0

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `boliboba` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `boliboba`


In [10]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/37.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DimaSK1/Qwen2-1.5B-bnb-4bit-kl-sft-base/commit/59b3499dad2d85b6a58c2779b75d68ce5772807a', commit_message='End of training', commit_description='', oid='59b3499dad2d85b6a58c2779b75d68ce5772807a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DimaSK1/Qwen2-1.5B-bnb-4bit-kl-sft-base', endpoint='https://huggingface.co', repo_type='model', repo_id='DimaSK1/Qwen2-1.5B-bnb-4bit-kl-sft-base'), pr_revision=None, pr_num=None)

# ОБУЧЕНИЕ ПОСЛЕДУЮЩИХ МОДЕЛЕЙ

In [2]:
%%capture
!pip install -q --upgrade transformers accelerate peft unsloth

In [2]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

output_dir = 'Qwen2-0.5B-bnb-4bit-sft-5' # название модели после чикла обучения

# model_name = "DimaSK1/gemma_2b_bnb_4bit_medical_base"
model_name = "DimaSK1/Qwen2-0.5B-bnb-4bit-sft-4"

model_new, tokenizer_new = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    # dtype = dtype,
    load_in_4bit = load_in_4bit,
) 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-03 11:03:42.065057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756897422.411150      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756897422.509080      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

Unsloth 2025.8.10 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [3]:
import pandas as pd
# OUTPUT_DIR = 'gemma_2b_bnb_sft_good_bad_1'
train_data_good = pd.read_excel("/kaggle/input/g4-gen-qwen0-5-sft/g4_data_gen_qwen0.5_sft.xlsx")
train_data_bad = pd.read_excel("/kaggle/input/g4-gen-qwen0-5-base/g4_data_gen_qwen0.5_base.xlsx")
# data = train_data
train_data = pd.concat([train_data_good, train_data_bad])
train_data = train_data[['description', 'result']]
train_data = train_data.sample(frac=1).reset_index(drop=True)
train_data

,description,result
0,пунктирован узел коллоидного в разной степени ...,Bethesda - категория II
1,п 38 в мазке неравномерной толщины с обильным...,В соответствии с критериями системы классифика...
2,Л12: в препарате на фоне коллоидаопределяются ...,Для каждого клетки фолликулярного эпителия в п...
3,Л 14 (№3424/22). Пунктирован узловой коллоидны...,Доброкачественное образование в соответствии с...
4,Л 14 (№4284/20). Пунктирован узловой коллоидын...,Доброкачественное образование в соответствии с...
...,...,...
995,"Пер 19 (№3258/20 1,2 - 2 мазка). На фоне элеме...",Доброкачественное образование в соответствии с...
996,П 17 (№3728/200. На фоне коллоида и измененных...,Доброкачественное образование в соответствии с...
997,П45 В мазке очень высокой клеточности неравно...,П45 В мазке очень высокой клеточности неравно...
998,л32 в мазке неравномерной толщины с обильным п...,В соответствии с критериями системы классифика...


In [4]:
import pandas as pd

train_data.to_excel('g4_train_qwen0.5_sft.xlsx', index=False)
# train_data1 = pd.read_excel('/kaggle/input/g1-train-qwen-tema/g1_train_qwen_tema.xlsx')
train_data1 = train_data[['description', 'result']]
train_data1

,description,result
0,пунктирован узел коллоидного в разной степени ...,Bethesda - категория II
1,п 38 в мазке неравномерной толщины с обильным...,В соответствии с критериями системы классифика...
2,Л12: в препарате на фоне коллоидаопределяются ...,Для каждого клетки фолликулярного эпителия в п...
3,Л 14 (№3424/22). Пунктирован узловой коллоидны...,Доброкачественное образование в соответствии с...
4,Л 14 (№4284/20). Пунктирован узловой коллоидын...,Доброкачественное образование в соответствии с...
...,...,...
995,"Пер 19 (№3258/20 1,2 - 2 мазка). На фоне элеме...",Доброкачественное образование в соответствии с...
996,П 17 (№3728/200. На фоне коллоида и измененных...,Доброкачественное образование в соответствии с...
997,П45 В мазке очень высокой клеточности неравно...,П45 В мазке очень высокой клеточности неравно...
998,л32 в мазке неравномерной толщины с обильным п...,В соответствии с критериями системы классифика...


In [5]:
X_train = train_data

alpaca_prompt = """
### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer_new.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples['description']
    outputs = examples['result']
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(X_train),
})

train_dataset_dict = train_dataset_dict.map(formatting_prompts_func, batched = True,)
train_dataset_dict['train'][1]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'description': 'п 38 в мазке  неравномерной толщины с обильным плотным коллоидом и с  большой примесью крови цитограмма характерная для коллоидного в разной степени пролиферирующего зоба с     участками аденоматоза и выраженными  дегенеративными изменениями.',
 'result': 'В соответствии с критериями системы классификации Bethesda: доброкачественное образование щитовидной железыдиагностическая категория II.',
 'text': '\n### Input:\nп 38 в мазке  неравномерной толщины с обильным плотным коллоидом и с  большой примесью крови цитограмма характерная для коллоидного в разной степени пролиферирующего зоба с     участками аденоматоза и выраженными  дегенеративными изменениями.\n\n### Output:\nВ соответствии с критериями системы классификации Bethesda: доброкачественное образование щитовидной железыдиагностическая категория II.<|endoftext|>'}

In [ ]:
import torch
from trl import SFTTrainer
from transformers.trainer import Accelerator
import warnings

class CustomSFTTrainer_w_kl(SFTTrainer):
    def __init__(self, model, args, train_dataset, dataset_text_field, data_collator=None, processing_class=None, kl_alpha=0.1):
        
        super().__init__(model=model,
                         args=args,
                         train_dataset=train_dataset,
                         dataset_text_field=dataset_text_field,
                         data_collator=data_collator,
                         processing_class=processing_class)

        if not hasattr(model, 'disable_adapter'):
            warnings.warn(
                "The model provided does not appear to be a PEFT model with a LoRA adapter. "
                "The KL divergence loss term will not be calculated and the trainer will fall back to standard SFT."
            )

        self.accelerator = Accelerator()
        self._total_train_tokens = 0
        # After super().__init__, self._metrics is initialized by the parent Trainer class.
        # We need to add our custom metric to it.
        if "kl_loss" not in self._metrics["train"]:
            self._metrics["train"]["kl_loss"] = []
        if "eval" in self._metrics and "kl_loss" not in self._metrics["eval"]:
            self._metrics["eval"]["kl_loss"] = []
        self.kl_alpha = kl_alpha


    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Computes SFT loss and adds a KL divergence penalty term.
        The reference log probabilities are computed on the fly by disabling the LoRA adapter.
        """
        if self.model_accepts_loss_kwargs:
            loss_kwargs = {}
            if num_items_in_batch is not None:
                loss_kwargs["num_items_in_batch"] = num_items_in_batch
        # Compute standard SFT loss with the adapter enabled
        sft_loss, outputs = super().compute_loss(model, inputs, return_outputs=True, **loss_kwargs)

        # If the model is not a PEFT model, we can't compute KL loss, so we return SFT loss
        if not hasattr(model, 'disable_adapter'):
            warnings.warn(
                "The model provided does not appear to be a PEFT model with a LoRA adapter. "
                "The KL divergence loss term will not be calculated and the trainer will fall back to standard SFT."
            )
            return (sft_loss, outputs) if return_outputs else sft_loss

        # Get the logits from the adapter-enabled model
        logits = outputs.logits
        labels = inputs["labels"]

        # Compute reference logits by disabling the adapter
        with torch.no_grad(), model.disable_adapter():
            ref_outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask", None),
            )
        ref_logits = ref_outputs.logits

        # Shift all tensors for next-token prediction
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_ref_logits = ref_logits[..., :-1, :].contiguous()

        # Create a mask for the answer tokens (non -100 labels)
        answer_mask = (shift_labels != -100).float()

        # Calculate log probabilities from the adapter model and reference model logits
        log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
        ref_log_probs = torch.nn.functional.log_softmax(shift_ref_logits, dim=-1)

        # Calculate KL divergence: KL(P_ref || P_model) = sum(P_ref * (log P_ref - log P_model))
        kl_divergence = torch.exp(ref_log_probs) * (ref_log_probs - log_probs)

        # Sum over the vocabulary dimension and apply the mask and average over the sequence and batch
        kl_loss_per_token = (kl_divergence.sum(dim=-1) * answer_mask)
        kl_loss = kl_loss_per_token.sum() / answer_mask.sum()

        # Combine losses
        loss = sft_loss + self.kl_alpha * kl_loss

        # Logging
        self._metrics["train"]["kl_loss"].append(kl_loss.item())

        return (loss, outputs) if return_outputs else loss


from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    # fp16 = False,
    # bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

trainer = CustomSFTTrainer_w_kl(
    model=model_new,
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    kl_alpha=0.1
)

In [6]:
import copy
import torch
import torch.nn.functional as F
from trl import SFTTrainer

class EMATeacherGPUTrainer(SFTTrainer):
    """
    EMA teacher trainer (GPU-only).
    - teacher is a deepcopy of the model, placed on the same device and cast to float32.
    - EMA updates only floating parameters by name (teacher_param = decay * teacher_param + (1-decay) * student_param).
    - Designed for non-sharded single-GPU (or standard DataParallel) runs.
    WARNING: doubles GPU memory usage.
    """
    def __init__(self, ema_decay=0.999, alpha=1.0, beta=0.05, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.ema_decay = float(ema_decay)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.temperature = float(temperature)
        self.teacher = None
        self._ema_initialized = False
        self._student_param_map = None  # name -> param (view into student model)

    def create_teacher(self):
        if self.teacher is not None:
            return

        # Deep-copy the student model and place teacher on same device as student.
        # Cast teacher to float32 for stable EMA.
        model_device = next(self.model.parameters()).device
        self.teacher = copy.deepcopy(self.model).to(model_device)

        # Cast teacher parameters to float32 (in-place)
        for p in self.teacher.parameters():
            if p.dtype != torch.float32:
                p.data = p.data.to(torch.float32)

        self.teacher.eval()
        for p in self.teacher.parameters():
            p.requires_grad = False

        # Build a mapping name -> student parameter for quick EMA updates.
        # Note: this holds references to the student parameters (they will change during training).
        self._student_param_map = dict(self.model.named_parameters())

        self._ema_initialized = True

    def _move_inputs_to_device(self, inputs, device):
        out = {}
        for k, v in inputs.items():
            if torch.is_tensor(v):
                out[k] = v.to(device)
            else:
                out[k] = v
        return out

    @torch.no_grad()
    def _update_ema(self):
        """
        Update teacher parameters with EMA from student.
        Only updates named parameters that exist in both teacher and student.
        Teacher params are assumed float32 and on the same device.
        """
        decay = float(self.ema_decay)
        if self.teacher is None or self._student_param_map is None:
            return

        # Iterate teacher named parameters and update from student param with same name
        for t_name, t_param in self.teacher.named_parameters():
            s_param = self._student_param_map.get(t_name, None)
            if s_param is None:
                # unexpected mismatch; skip
                continue

            # only update floating point parameters
            if not torch.is_floating_point(t_param):
                continue

            # read student param as float32 on teacher device
            s_data = s_param.data.detach().to(torch.float32).to(t_param.device)

            # EMA: t = decay * t + (1 - decay) * s
            # t_param is float32; do in-place ops on t_param.data
            t_param.data.mul_(decay)
            t_param.data.add_(s_data * (1.0 - decay))

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
      # Ensure teacher initialized
      if not self._ema_initialized:
          self.create_teacher()

      model_device = next(model.parameters()).device
      # Prepare inputs for student (do not mutate original)
      inputs_for_student = {k: (v.to(model_device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
      if "labels" not in inputs_for_student:
          raise RuntimeError("No 'labels' in inputs_for_student — check data collator/tokenization")
      labels = inputs_for_student.pop("labels")  # on model_device

      # Student forward
      outputs = model(**inputs_for_student, labels=labels)
      ce_loss = outputs.loss
      logits = outputs.logits  # [B,S,V] on model_device

      # Teacher forward (teacher is on same device in EMATeacherGPUTrainer)
      teacher_device = next(self.teacher.parameters()).device
      teacher_inputs = {k: (v.to(teacher_device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
      teacher_inputs["use_cache"] = False
      with torch.no_grad():
          teacher_logits = self.teacher(**teacher_inputs).logits # [B,S,V] on teacher_device
          # print(not torch.isfinite(teacher_logits).all())
        
      # # --- Если teacher_logits = None → создаём фиктивный тензор NaN ---
      # if teacher_logits is None:
      #     B, L = inputs["input_ids"].shape
      #     V = model.config.vocab_size
      #     teacher_logits = torch.full((B, L, V), float("nan"), device=model.device)
        
      #   # --- Стабилизируем ---
      #     teacher_logits = torch.nan_to_num(
      #           teacher_logits,
      #           nan=0.0,
      #           posinf=1e4,
      #           neginf=-1e4,
      #       )
      # --- Diagnostics: check finite-ness ---
      if not torch.isfinite(teacher_logits).all():
        # print("[WARN] teacher_logits contains non-finite values (NaN/Inf). Replacing with zeros for stability.")

        # безопасно получаем min/max, игнорируя NaN
        finite_mask = torch.isfinite(teacher_logits)
        if finite_mask.any():
            t_min = teacher_logits[finite_mask].min().item()
            t_max = teacher_logits[finite_mask].max().item()
        else:
            t_min, t_max = float('nan'), float('nan')

        # print(f" teacher_logits stats (finite only): min={t_min:.6g}, max={t_max:.6g}")

        teacher_logits = torch.nan_to_num(
            teacher_logits,
            nan=0.0,
            posinf=1e4,
            neginf=-1e4
        )

      # Masking valid tokens
      valid_mask = labels.ne(-100)  # [B, S]
      num_valid = int(valid_mask.sum().item())
      if num_valid == 0:
          raise ValueError("No valid tokens in labels (all -100). Stop training and fix data.")

      # Select valid positions and cast to float32 for KL computation
      student_logits_sel = logits[valid_mask].float()         # [N_valid, V] on model_device
      teacher_logits_sel = teacher_logits.to(logits.device)[valid_mask].float()  # ensure same device

      T = float(self.temperature)
      V = student_logits_sel.size(-1)

      # Stable teacher log-probs and probs
      teacher_logp = F.log_softmax(teacher_logits_sel / T, dim=-1)  # may contain large negative numbers
      # compute teacher_p as exp(teacher_logp) in float32
      teacher_p = torch.exp(teacher_logp)

      # Option 1: clamp teacher_p to avoid exact zeros (recommended)
      eps = 1e-12
      teacher_p = teacher_p.clamp(min=eps)
      # recompute teacher_logp from clamped teacher_p to keep consistency
      teacher_logp = torch.log(teacher_p)

      # Optionally apply smoothing to teacher distribution (helps if teacher is extremely peaky)
      smoothing = 0.0  # set small value like 1e-6..1e-4 to try; here default 0
      if smoothing > 0.0:
          # teacher_p <- (1 - s) * teacher_p + s / V
          teacher_p = teacher_p * (1.0 - smoothing) + (smoothing / float(V))
          teacher_logp = torch.log(teacher_p.clamp(min=eps))

      # student_logp (already stable because log_softmax)
      student_logp = F.log_softmax(student_logits_sel / T, dim=-1)

      # Compute per-element KL contribution: teacher_p * (teacher_logp - student_logp)
      # This is numerically more explicit than relying on F.kl_div under extreme tails.
      per_token_per_vocab = teacher_p * (teacher_logp - student_logp)  # shape [N_valid, V]

      # Replace NaN/Infs in contributions just in case (shouldn't happen after clamp, but safe)
      per_token_per_vocab = torch.nan_to_num(per_token_per_vocab, nan=0.0, posinf=0.0, neginf=0.0)

      # Sum over vocab then over tokens
      kl_sum = per_token_per_vocab.sum()  # scalar
      kl_sum = kl_sum * (T ** 2)

      kl_loss = kl_sum / float(num_valid)

      # If kl_loss is still nan (very unlikely), fallback to small value and warn
      if not torch.isfinite(kl_loss):
          print("[WARN] kl_loss is non-finite after stabilization; replacing with 0.0")
          kl_loss = torch.tensor(0.0, device=ce_loss.device, dtype=ce_loss.dtype)

      total = self.alpha * ce_loss + self.beta * kl_loss

      # Debug prints
      ce_f = float(ce_loss.detach().cpu().item())
      kl_f = float(kl_loss.detach().cpu().item())
      total_f = float(total.detach().cpu().item())
      # print(f"[TRAIN-STEP DEBUG] ce_loss={ce_f:.6g}, kl_loss={kl_f:.6g}, total={total_f:.6g}, num_valid={num_valid}")

      # EMA update
      try:
          with torch.no_grad():
              self._update_ema()
      except Exception as e:
          print(f"[WARN] EMA update failed: {e}")

      # Log with trainer
      try:
          self.log({"train/ce_loss": ce_f, "train/kl_loss": kl_f, "train/total_loss": total_f, "train/num_valid": num_valid})
      except Exception:
          pass

      return (total, outputs) if return_outputs else total


from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = False,
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

# Инициализация и запуск
trainer = EMATeacherGPUTrainer(
    model=model_new,  # PeFT модель с LoRA
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    tokenizer=tokenizer_new,
    ema_decay=0.999,
    alpha=1.0, beta=0.05, temperature=1.0
)


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

## Тренер с комбинированным лоссом для обучения

In [15]:
from trl import SFTTrainer
import torch
import torch.nn as nn


class CombinedLossSFTTrainer(SFTTrainer):
    def __init__(self, alpha=0.7, beta=0.3, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha
        self.beta = beta
        self.temperature = temperature
        # Сохраняем исходные веса модели для teacher
        self.original_state = {k: v.detach().clone() for k, v in self.model.state_dict().items()}
        self.teacher = None

    def create_teacher(self):
        # Создаём единожды глубокую копию текущей модели как teacher
        if self.teacher is None:
            import copy
            self.teacher = copy.deepcopy(self.model)
            self.teacher.to(self.model.device)
            self.teacher.eval()
            for p in self.teacher.parameters():
                p.requires_grad = False

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Обеспечиваем teacher
        self.create_teacher()
        labels = inputs.pop('labels')
        # Forward студента
        outputs = model(**inputs, labels=labels)
        ce_loss = outputs.loss
        logits = outputs.logits
        # Forward teacher без градиентов
        with torch.no_grad():
            teacher_logits = self.teacher(**inputs).logits
        # KL-дивергенция
        student_lp = nn.functional.log_softmax(logits / self.temperature, dim=-1)
        teacher_p = nn.functional.softmax(teacher_logits / self.temperature, dim=-1)
        kl_loss = nn.functional.kl_div(
            student_lp, teacher_p, reduction='batchmean'
        ) * (self.temperature ** 2)
        total = self.alpha * ce_loss + self.beta * kl_loss
        return (total, outputs) if return_outputs else total

from transformers import TrainingArguments
# Аргументы тренировки
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    # per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps = 2,
    warmup_steps = 5,
    num_train_epochs = 4,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_torch",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    report_to = "none" # Use this for WandB etc
)

alpha = 1.0
beta = 0.05
temperature = 1.0

# Инициализация и запуск
trainer = CombinedLossSFTTrainer(
    model=model_new,
    args=training_args,
    train_dataset=train_dataset_dict['train'],
    dataset_text_field = "text",
    # eval_dataset=eval_dataset,
    # data_collator=default_data_collator,
    tokenizer=tokenizer_new,
    alpha=alpha,
    beta=beta,
    temperature=temperature
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

## Стандартный тренер

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model_new,
    tokenizer = tokenizer_new,
    train_dataset = train_dataset_dict['train'],
    dataset_text_field = "text",
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
import os


os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 4 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,399,104 of 498,431,872 (0.88% trained)


Step,Training Loss,entropy
10,0.340600,0
20,0.288000,No Log
30,0.245600,No Log
40,0.351000,No Log
50,0.363800,No Log
60,0.322900,No Log
70,0.324100,No Log
80,0.352400,No Log
90,0.295600,No Log
100,0.328000,No Log


In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")

In [9]:
!huggingface-cli login --token $secret_value_0

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `boliboba` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `boliboba`


In [10]:
trainer.push_to_hub()

  0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DimaSK1/Qwen2-0.5B-bnb-4bit-sft-5/commit/6f2c58d315c2398e7998af33fea4552042fb8aeb', commit_message='End of training', commit_description='', oid='6f2c58d315c2398e7998af33fea4552042fb8aeb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DimaSK1/Qwen2-0.5B-bnb-4bit-sft-5', endpoint='https://huggingface.co', repo_type='model', repo_id='DimaSK1/Qwen2-0.5B-bnb-4bit-sft-5'), pr_revision=None, pr_num=None)

# Генерация ответов для обучения

In [1]:
%%capture
!pip install -q --upgrade unsloth transformers accelerate peft

In [2]:
import torch
# import os
# os.environ["UNSLOTH_DISABLE_RL_PATCH"] = "1"
from unsloth import FastLanguageModel


max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model_name = "unsloth/Qwen2-0.5B-bnb-4bit"
# model_name = "DimaSK1/Qwen2-1.5B-bnb-4bit_1"
model_name = 'DimaSK1/Qwen2-0.5B-bnb-4bit-kl-sft-base'



model_new, tokenizer_new = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-10 13:33:35.046109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757511215.376607      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757511215.472818      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


AttributeError: module 'transformers.models.bit.modeling_bit' has no attribute 'Linear'

In [5]:
import pandas as pd


test_data1 = pd.read_excel('/kaggle/input/dataf1/data 1.xlsx')
# test_data1= test_data1[:100]
# test_data1 = test_data1[950:]

In [6]:
alpaca_prompt = """
### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer_new.eos_token # Must add EOS_TOKEN

def prompts_func(examples):
    inputs = examples["description"]
    texts = []
    for input in inputs:
        output = ''
        text = alpaca_prompt.format(input, output)
        texts.append(text)
    return { "text" : texts, }


X_test = test_data1[['description', 'target']]
text_data = prompts_func(X_test)
test_data = pd.DataFrame(data=text_data)
target = X_test['target'].to_list()

num_samples = test_data.shape[0]
num_samples

1000

In [5]:
test_data['text'][0] 

'\n### Input:\nП 20 (№7053/22 1). Пунктирован коллоидный зоб с морфологическими признаками регрессивных изменений и небольшим содержанием в мазке дистрофичных тиреоцитов на фоне коллоида и измененных эритроцитов. Л 20 (№7053/22 2). Пунктирован узловой коллоидный зоб с кистозно-регрессивными изменениямиэлементами воспалительной инфильтрации и небольшим содержанием в мазке дистрофичных клеток фолликулярного эпителия щитовидной железы.\n\n### Output:\n'

In [7]:
from tqdm import tqdm
import torch
from unsloth import FastLanguageModel
import numpy as np



# Убедитесь, что модель переведена в режим инференса
FastLanguageModel.for_inference(model_new)

results = []
perplexities1 = []
perplexities2 = []
token_probabilities = {}

# Функция для вычисления перплексии и вероятностей для сгенерированного текста
def calculate_perplexity_and_generated_probabilities(model, tokenizer, test_data, num_samples=1000):
    model.eval()  # Переводим модель в режим оценки
    total_log_prob = 0.0
    total_words = 0

    with torch.no_grad():
        for i in tqdm(range(num_samples)):
            tkn_probs = []
            tkn_distr_probs = []
            # Получаем токенизированный текст
            inputs = tokenizer(test_data['text'][i], return_tensors='pt', truncation=True, max_length=1024).to('cuda')

            # Генерация текста с возвратом вероятностей
            output = model.generate(
                **inputs,
                max_new_tokens=128,
                use_cache=False,
                # output_scores=True,
                # return_dict_in_generate=True
            )
            
            generated_ids = output[:, inputs.input_ids.shape[-1]:]  # Только сгенерированные токены
            # scores = output["scores"]  
            
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            results.append(generated_text)

            # # Сохраняем вероятности для сгенерированных токенов
            # for step, (logits, token_id) in enumerate(zip(scores, generated_ids[0])):
            #     token_probs = torch.softmax(logits[0], dim=-1).cpu().numpy() # Вероятности для всех токенов
            #     token_distribution = {
            #         token_id: {tokenizer.decode([token_id]): prob}
            #         for token_id, prob in enumerate(token_probs)
            #     }
            #     tkn_probs.append({
            #         "token": tokenizer.decode([token_id.item()]),
            #         "probability": token_probs[token_id].item()
            #     })
            #     if i == 0:
            #         # Токен ID и его вероятность
            #         token_id = generated_ids[0, step].item()
                    
            #         token_prob = token_probs[token_id].item()
                    
            #         # print(step, tokenizer.decode(token_id), token_probs.max())
            #         # Сохраняем токен, его вероятность и всё распределение
            #         tkn_distr_probs.append({
            #             "step": step,
            #             "token_id": token_id,
            #             "token": tokenizer.decode([token_id]),
            #             "probability": token_prob,
            #             "distribution": token_distribution  # Сопоставление токенов и вероятностей
                    # })

            # token_probabilities[i] = tkn_distr_probs 
            # if len(tkn_probs) != len(scores):
            #     print(len(tkn_probs), len(scores))

            # log_probs_for_generated = [
            #     np.log(item["probability"])
            #     for item in tkn_probs
            #     if item["probability"] > 0
            # ]

            # if len(log_probs_for_generated) > 0:  
            #     total_log_prob = sum(log_probs_for_generated)
            #     total_words = len(log_probs_for_generated)

            # if total_words > 0:
            #     perplexities1.append(np.exp(-total_log_prob / total_words))
            
    return perplexities1, token_probabilities, results


ppl1, token_probs, results = calculate_perplexity_and_generated_probabilities(model_new, tokenizer_new, test_data, 
                                               num_samples=num_samples)

100%|██████████| 1000/1000 [1:01:16<00:00,  3.68s/it]


In [8]:
result_data = pd.DataFrame({'description': test_data1['description'],
                            'result' : results, 'target' : target})
result_data.to_excel('g0_data_gen_qwen1.5_kl-sft.xlsx', index=False)
result_data

,description,result,target
0,П 20 (№7053/22 1). Пунктирован коллоидный зоб ...,Доброкачественные образования обеих долей в с...,Доброкачественные образования обеих долей в с...
1,Пер 11 материал представлен содержимым кистозн...,В соответствии с критериями системы классифи...,В соответствии с критериями системы классифи...
2,В препарате на фоне кистозно-геморрагического ...,Доброкачественное образование в соответствии с...,Доброкачественное образование в соответствии с...
3,л3 В мазках очень высокой клеточности обнаруже...,В соответствии с критериями классификационной...,В соответствии с критериями классификационной...
4,п5 в мазке с гетерогенным клеточным составом н...,В соответствии с классификационной системой ...,В соответствии с классификационной системой ...
...,...,...,...
995,П17; Л17: в препаратах на фоне коллоидаопредел...,В соответствии с критериями Bethesda: доброкач...,В соответствии с критериями Bethesda: доброкач...
996,пунктирован узел коллоидного в разной степени ...,Bethesda - категория II,Bethesda - категория II
997,П4 Материал малоклеточныйпредставлен содержимы...,В соответствии с критериями системы классифика...,В соответствии с классификационной системой ...
998,материал недостаточно информативный : в мазках...,Bethesda - категория I,Bethesda - категория I


In [3]:
test_data1

,Unnamed: 0,target,description
0,11000,соответствии критериями системы образование ди...,п9 В мазке цитологическая картина характерная ...
1,11001,В соответствии с классификационной системой Be...,Л 3 В мазках неравномерной толщины с очень бол...
2,11002,В соответствии с критериями системы классифи...,л6 материал представлен кистозно-геморрагическ...
3,11003,В соответствии с критериями системы классифика...,п13 В мазке неравномерной толщины с обильным п...
4,11004,В соответствии с критериями системы классифика...,Л1 материал с большой примесью периферической ...
...,...,...,...
95,11095,Цитограмма новообразования из фолликулярных кл...,п22 в мазках на фоне кистозно-геморрагических ...
96,11096,Доброкачественные образования в соответствии ...,П 32 (№1592/22 1). Мазок интенсивно разбавлен ...
97,11097,Доброкачественные образования обеих долей в с...,П 6 (№5537/20 1). Мазок интенсивно разбавлен п...
98,11098,В соответствии с критериями Bethesda: доброкач...,Л23 В мазке на фоне коллоида эритроцитовмакро...


In [2]:
import pandas as pd
import torch
from unsloth import FastLanguageModel
from tqdm import tqdm
import numpy as np


# Функция для вычисления перплексии и вероятностей для сгенерированного текста
def calculate_perplexity_and_generated_probabilities(model, tokenizer, data):
    alpaca_prompt = """
    ### Input:
    {}
    
    ### Output:
    {}"""
    
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    
    def prompts_func(examples):
        inputs = examples["description"]
        texts = []
        for input in inputs:
            output = ''
            text = alpaca_prompt.format(input, output)
            texts.append(text)
        return { "text" : texts, }
    
    
    X_test = data[['description', 'target']]
    text_data = prompts_func(X_test)
    test_data = pd.DataFrame(data=text_data)
    target = X_test['target'].to_list()
    
    num_samples = test_data.shape[0]
    
    results = []
    perplexities1 = []
    perplexities2 = []
    token_probabilities = {}
    model.eval()  # Переводим модель в режим оценки
    total_log_prob = 0.0
    total_words = 0
    with torch.no_grad():
        for i in tqdm(range(num_samples)):
            tkn_probs = []
            tkn_distr_probs = []
            # Получаем токенизированный текст
            inputs = tokenizer(test_data['text'][i], return_tensors='pt', truncation=True, max_length=1024).to('cuda')

            # Генерация текста с возвратом вероятностей
            output = model.generate(
                **inputs,
                max_new_tokens=64,
                use_cache=False,
                # output_scores=True,
                pad_token_id=tokenizer.eos_token_id  # Убедитесь, что pad_token_id установлен
            )
            
            # generated_ids = output["sequences"][:, inputs.input_ids.shape[-1]:]  # Только сгенерированные токены
            # scores = output["scores"]  
            generated_ids = output[:, inputs['input_ids'].shape[-1]:]
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            results.append(generated_text)
            # print(generated_text)
    
    return results, target



test_data1 = pd.read_excel('/kaggle/input/dataf11/data 11.xlsx')[:100]

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model_name = "unsloth/Qwen2-1.5B-bnb-4bit"
# model_name = "DimaSK1/Qwen2-1.5B-bnb-4bit_1"
for i in range(6):
    model_name = f'DimaSK1/Qwen2-1.5B-bnb-4bit_ema_{i}'
    if i == 0:
        model_name = 'DimaSK1/Qwen2-1.5B-bnb-4bit_base_ema'
        
    model_new, tokenizer_new = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,
    ) # Enable native 2x faster inference
    
    FastLanguageModel.for_inference(model_new)
    
    results, target = calculate_perplexity_and_generated_probabilities(model_new, tokenizer_new, test_data1)
    
    result_data = pd.DataFrame({'description': test_data1['description'],
                            'result' : results, 'target' : test_data1['target']})
    result_data.to_excel(f'g{i}_data_test_qwen0.5_sft.xlsx', index=False)

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

# Получение метрик

In [ ]:
%%capture
!pip install -q --upgrade transformers accelerate peft unsloth
!pip install rouge-score -q

In [ ]:
import pandas as pd
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import AutoTokenizer
from sklearn.metrics import mean_absolute_error


# Функция для вычисления BLEU
def calculate_bleu(reference, candidate):
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()
    return sentence_bleu(reference_tokens, candidate_tokens)

# Функция для вычисления ROUGE
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

# Функция для вычисления BLEU с использованием токенайзера
def calculate_bleu_with_tokenizer(candidate, reference, tokenizer):
    candidate_tokens = tokenizer.tokenize(candidate)
    reference_tokens = [tokenizer.tokenize(reference)]  # Ожидается список списков для BLEU
    return sentence_bleu(reference_tokens, candidate_tokens)

# Функция для вычисления ROUGE с использованием токенайзера
def calculate_rouge_with_tokenizer(candidate, reference, tokenizer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    candidate_decoded = tokenizer.decode(tokenizer(candidate)["input_ids"], skip_special_tokens=True)
    reference_decoded = tokenizer.decode(tokenizer(reference)["input_ids"], skip_special_tokens=True)
    scores = scorer.score(reference_decoded, candidate_decoded)
    return scores

# Загружаем русский автотокенайзер
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

df = pd.read_excel('/kaggle/input/g1-gen-qwen-klsft-opt1/g1_data_gen_qwen-klsft_opt.xlsx')[['result', 'target']]
# df = result_data

# Итерация по строкам датафрейма и вычисление метрик
results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    candidate = row['result']
    reference = row['target']
    
    bleu_score = calculate_bleu_with_tokenizer(candidate, reference, tokenizer)
    rouge_scores = calculate_rouge_with_tokenizer(candidate, reference, tokenizer)

    bleu_notoken = calculate_bleu(reference, candidate)
    rouge_notoken1, rouge_notoken2, rouge_notokenL = calculate_rouge(reference, candidate)
    
    rouge1 = rouge_scores['rouge1'].fmeasure
    rouge2 = rouge_scores['rouge2'].fmeasure
    rougeL = rouge_scores['rougeL'].fmeasure
    
    results.append({
        "BLEU": bleu_score,
        "BLEU_NOTOKEN": bleu_notoken,
        "ROUGE-1": rouge1,
        "ROUGE1_NOTOKEN": rouge_notoken1,
        "ROUGE-2": rouge2,
        "ROUGE2_NOTOKEN": rouge_notoken2,
        "ROUGE-L": rougeL,
        "ROUGEL_NOTOKEN": rouge_notokenL
    })

# Создаем датафрейм с результатами
results_df = pd.DataFrame(results)
# print(results_df)
results_df.to_excel('Метрики_qwen_klsft_0_opt.xlsx', index=False)

## BLUERT, METEOR и другие

In [ ]:
%%capture
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip
!pip install nltk sacrebleu moverscore bert-score --upgrade

In [ ]:
import pandas as pd
import nltk
from nltk.translate.meteor_score import meteor_score
from sacrebleu.metrics import CHRF
from bleurt import score as bleurt_score
# from moverscore_v2 import get_idf_dict, word_mover_score
import bert_score
from tqdm import tqdm
import re


# Загружаем токенизатор для NLTK
nltk.download('punkt')
nltk.download('wordnet')

# Инициализация метрик
chrf = CHRF()
bleurt = bleurt_score.BleurtScorer("/content/bleurt/BLEURT-20")   # скачай модель заранее: https://github.com/google-research/bleurt

# ==== ФУНКЦИИ ====
def calculate_metrics(reference, candidate):
    """Вычисляем все метрики для пары текстов"""

    # METEOR
    meteor = meteor_score([reference.split()], candidate.split())

    # ChrF
    chrf_score = chrf.sentence_score(candidate, [reference]).score

    # BLEURT
    bleurt_val = bleurt.score(references=[reference], candidates=[candidate])[0]

    # MoverScore
    # idf_dict_ref = get_idf_dict([reference])
    # idf_dict_cand = get_idf_dict([candidate])
    # mover = word_mover_score([reference], [candidate],
    #                          idf_dict_ref, idf_dict_cand,
    #                          stop_words=[], n_gram=1, remove_subwords=True)[0]

    # BERTScore (используем модель для русского — 'xlm-roberta-large')
    P, R, F1 = bert_score.score([candidate], [reference], lang="ru", model_type="xlm-roberta-large")
    bert_f1 = F1.mean().item()

    return {
        "METEOR": meteor,
        "ChrF": chrf_score,
        "BLEURT": bleurt_val,
        # "MoverScore": mover,
        "BERTScore_F1": bert_f1
    }

# ==== НОРМАЛИЗАЦИЯ ====
def normalize_text(text: str) -> str:
    """Приведение текста к единому виду"""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)   # убираем лишние пробелы
    text = re.sub(r"[^\w\sа-яё]", "", text)  # оставляем только буквы и цифры
    return text


def evaluate_file(filepath, output_path):
    """Считает метрики для Excel-файла с колонками result/target"""
    df = pd.read_excel(filepath)[["result", "target"]]

    # нормализация
    df["result"] = df["result"].apply(normalize_text)
    df["target"] = df["target"].apply(normalize_text)

    results = []
    print(f'Обработка {filepath}...')
    for i in tqdm(range(df.shape[0])):
      row = df.iloc[i]

      cand = str(row.iloc[0])
      ref = str(row.iloc[1])
      scores = calculate_metrics(ref, cand)
      results.append(scores)

    results_df = pd.DataFrame(results)

    # Сохраняем только построчные метрики, без среднего
    results_df.to_excel(output_path, index=False)
    print(f"[+] Файл сохранён: {output_path}")

# ==== ПРИМЕР ЗАПУСКА ====
files = [f"/content/bleurt/test_data/g{i}_data_test_qwen_tema.xlsx" for i in range(4, 8)]
for i, f in enumerate(files):
    evaluate_file(f, f"Метрики_1_qwen_test_tema_{i}.xlsx")

## Визуализация метрик

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Ввод параметров
methods = ['tema', 'sft', 'klsft']
num_generations = 7
trh = 0.9

print(f"Анализируемые методы: {methods}")
print(f"Количество поколений: {num_generations}")
print(f"Пороговое значение: {trh}")

# Инициализация структур данных
metrics = ["BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L"]
data_values = {metric: {method: [] for method in methods} for metric in metrics}
data_stds = {metric: {method: [] for method in methods} for metric in metrics}

# Цикл по методам и поколениям
for method in methods:
    for gen in range(num_generations):
        file_path = Path(f'Метрики_{method}_{gen}.xlsx')
        
        if file_path.exists():
            try:
                df = pd.read_excel(file_path, engine='openpyxl')
                # print(f"✅ Загружен файл: {file_path}")
                
                # Обработка каждой метрики
                for metric in metrics:
                    # Поиск правильного названия столбца
                    col_name = None
                    for variant in [metric.upper(), metric, metric.capitalize()]:
                        if variant in df.columns:
                            col_name = variant
                            break
                    
                    if col_name is None:
                        print(f"⚠️ Столбец '{metric}' не найден в {file_path}")
                        data_values[metric][method].append(0)
                        data_stds[metric][method].append(0)
                        continue
                    
                    # Расчет метрик
                    values = df[col_name]
                    above_threshold = values[values > trh].count() / len(values)
                    std_dev = values.std()
                    
                    data_values[metric][method].append(above_threshold)
                    data_stds[metric][method].append(std_dev)
            
            except Exception as e:
                print(f"⛔ Ошибка при обработке {file_path}: {str(e)}")
                for metric in metrics:
                    data_values[metric][method].append(0)
                    data_stds[metric][method].append(0)
        else:
            print(f"⚠️ Файл не найден: {file_path}")
            for metric in metrics:
                data_values[metric][method].append(0)
                data_stds[metric][method].append(0)

# Настройки визуализации
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(14, 8))

# Цвета для методов
method_colors = plt.cm.tab10(np.linspace(0, 1, len(methods)))

# Маркеры для метрик
markers = ['o', 's', 'D', '^']
metric_markers = {metric: marker for metric, marker in zip(metrics, markers)}

# Генерация оси X (поколения)
x = np.arange(num_generations)

# Построение графиков
for method_idx, method in enumerate(methods):
    color = method_colors[method_idx]
    
    for metric in metrics:
        y = data_values[metric][method]
        y_err = data_stds[metric][method]
        marker = metric_markers[metric]
        
        # Линия для метода (один раз для каждой метрики)
        plt.plot(x, y, 
                 color=color, 
                 linestyle='-', 
                 alpha=0.7, 
                 zorder=1)
        
        # Точки с ошибками
        plt.errorbar(x, y, yerr=y_err,
                     fmt=marker,
                     color=color,
                     ecolor=color,
                     markersize=10,
                     capsize=4,
                     capthick=1.5,
                     elinewidth=1.5,
                     label=f'{method} - {metric}' if method_idx == 0 else "",
                     zorder=2)

# Настройка осей и заголовка
plt.xticks(x, [f'Поколение {i}' for i in range(num_generations)], fontsize=12)
plt.yticks(np.linspace(0, 1.0, 11), fontsize=10)
plt.ylabel("Доля значений > порога", fontsize=14)
plt.xlabel("Номер поколения", fontsize=14)
plt.title(f"Сравнение метрик по поколениям (порог = {trh})", fontsize=16, fontweight='bold')
plt.ylim(-0.05, 1.05)
plt.grid(True, linestyle='--', alpha=0.7)

# Создание кастомной легенды
legend_elements = []

# Добавление элементов для методов (цветные линии)
for method_idx, method in enumerate(methods):
    legend_elements.append(Line2D([0], [0], 
                                 color=method_colors[method_idx], 
                                 lw=3,
                                 label=method))

# Добавление элементов для метрик (маркеры)
for metric, marker in metric_markers.items():
    legend_elements.append(Line2D([0], [0], 
                                 marker=marker, 
                                 color='w', 
                                 markerfacecolor='black',
                                 markersize=10,
                                 label=metric.upper()))

# Размещение легенды
plt.legend(handles=legend_elements, 
           loc='upper center', 
           bbox_to_anchor=(0.5, -0.12),
           ncol=len(methods) + 1,  # +1 для метрик
           fontsize=10,
           frameon=True,
           shadow=True)

# Сохранение и отображение
plt.tight_layout()
filename = f'metrics_comparison_{trh}.jpeg'
plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f"✅ График сохранен как: {filename}")
plt.show()

# Оценка модели на бенчмарке MMLU

In [1]:
%%capture
!pip install -q --upgrade transformers accelerate peft unsloth

In [2]:
import os
import re
import time
import random
import json
import dataclasses
from dataclasses import dataclass
from typing import List, Dict, Optional, Any
from tqdm import tqdm

import torch
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel

# === Конфигурация по умолчанию ===
MEDICAL_SUBJECTS = ['all']
# MEDICAL_SUBJECTS = [
#     "anatomy",
#     "clinical_knowledge",
#     "college_biology",
#     # "college_medicine",
#     "medical_genetics",
#     "nutrition",
#     # "professional_medicine",
#     "virology",
#     "philosophy",
#     "formal_logic",
#     "global_facts",
#     "public_relations"
# ]
# 'abstract_algebra', 'all', 'anatomy', 'astronomy', 'auxiliary_train', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

USER_TPL = (
    "Вопрос: {question}\n"
    "Варианты ответа:\n"
    "A) {A}\nB) {B}\nC) {C}\nD) {D}\n\n"
    "Ответ:"
)

FEW_SHOT_TPL = (
    "Вопрос: {question}\n"
    "Варианты ответа:\n"
    "A) {A}\nB) {B}\nC) {C}\nD) {D}\n"
    "Правильный ответ: {answer}\n\n"
)

LETTER_TO_INDEX = {"A": 0, "B": 1, "C": 2, "D": 3}
INDEX_TO_LETTER = {v: k for k, v in LETTER_TO_INDEX.items()}

# === Вспомогательные функции ===
def normalize_answer_field(example: Dict[str, Any]) -> Dict[str, Any]:
    q = example.get("question") or example.get("input") or example.get("prompt") or ""
    choices = example.get("choices") or example.get("options")
    # print(choices)
    if not choices:
        candidates = [example.get(k) for k in ["A", "B", "C", "D"]]
        if all(isinstance(x, str) and x for x in candidates):
            choices = candidates
    assert choices and len(choices) == 4, f"Некорректные варианты: {choices}"

    ans = example.get("answer") or example.get("target")
    # print(ans)
    if isinstance(ans, str):
        ans = ans.strip()
        if ans in LETTER_TO_INDEX:
            gold_idx = LETTER_TO_INDEX[ans]
        else:
            try:
                gold_idx = choices.index(ans)
            except ValueError:
                gold_idx = None
    elif isinstance(ans, int):
        gold_idx = ans
    elif ans is None:
        gold_idx = 0
    assert gold_idx in [0, 1, 2, 3], f"Некорректный ответ: {ans}, {q}"
    return {"question": q, "choices": choices, "gold": gold_idx}


def build_few_shot_block(few_shot_examples: List[Dict[str, Any]]) -> str:
    return "".join(
        FEW_SHOT_TPL.format(
            question=ex["question"], A=ex["choices"][0], B=ex["choices"][1],
            C=ex["choices"][2], D=ex["choices"][3], answer=INDEX_TO_LETTER[ex["gold"]]
        )
        for ex in few_shot_examples
    )


def build_prompt(example: Dict[str, Any], few_shot_block: str = "") -> str:
    A, B, C, D = example["choices"]
    user = USER_TPL.format(question=example["question"], A=A, B=B, C=C, D=D)
    if few_shot_block:
        return ("Ты увидешь несколько решенных вопросов с правильным ответом.\n\n"
                + few_shot_block
                + "Далее ответь на следующий вопрос.\n\n"
                + user)
    return user


def extract_letter(text: str) -> Optional[str]:
    if not text:
        return None
    m = re.search(r"\b([A-D])\b", text.strip())
    if m:
        return m.group(1)
    m = re.search(r"([A-D])\)?\s*$", text.strip())
    return m.group(1) if m else None

# === Генерация с моделью Unsloth ===
def generate(prompt: str, max_new_tokens: int = 128) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, use_cache=False)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):]

@dataclass
class EvalConfig:
    subjects: List[str] = dataclasses.field(default_factory=lambda: MEDICAL_SUBJECTS)
    k_shot: int = 5
    limit_per_subject: Optional[int] = None
    seed: int = 42
    sleep: float = 0.0

@dataclass
class EvalResult:
    subject: str
    n: int
    correct: int
    accuracy: float
    eval_time: float

def load_subject(subject: str) -> Dataset:
    return load_dataset("cais/mmlu", subject)

def pick_few_shots(ds_valid: Optional[Dataset], k: int, rng: random.Random) -> List[Dict[str, Any]]:
    if not ds_valid or len(ds_valid) == 0 or k <= 0:
        return []
    idxs = list(range(len(ds_valid)))
    rng.shuffle(idxs)
    return [normalize_answer_field(ds_valid[i]) for i in idxs[:k]]

def evaluate_subject(subject: str, cfg: EvalConfig, rng: random.Random) -> EvalResult:

    raw = load_subject(subject)
    test_split = raw.get("test", 0)
    valid_split = raw.get("validation", 0) or raw.get("dev", 0)
    if test_split == 0 or valid_split == 0:
        return None

    few_shot_block = build_few_shot_block(pick_few_shots(valid_split, cfg.k_shot, rng))

    n_total, n_correct = 0, 0
    records = []
    start_time = time.time()
    if cfg.limit_per_subject is not None:
        lim = min(cfg.limit_per_subject, len(test_split))
    else:
        lim = len(test_split)
    for i in tqdm(range(lim)):
        ex = normalize_answer_field(test_split[i])
        prompt = build_prompt(ex, few_shot_block)
        print(prompt)
        raw_out = generate(prompt)
        pred_letter = extract_letter(raw_out) or ""
        pred_idx = LETTER_TO_INDEX.get(pred_letter, -1)
        correct = int(pred_idx == ex["gold"])
        n_total += 1
        n_correct += correct
        records.append({
            "subject": subject,
            "index": i,
            "question": ex["question"],
            "A": ex["choices"][0], "B": ex["choices"][1],
            "C": ex["choices"][2], "D": ex["choices"][3],
            "gold_letter": INDEX_TO_LETTER[ex["gold"]],
            "prediction_raw": raw_out.strip(),
            "prediction_letter": pred_letter,
            "correct": correct
        })
        if cfg.sleep:
            time.sleep(cfg.sleep)
    end_time = time.time()

    eval_time = end_time - start_time
    acc = n_correct / max(1, n_total)

    # os.makedirs("/kaggle/working/mmlu_med_logs", exist_ok=True)
    # import pandas as pd
    # pd.DataFrame.from_records(records).to_excel(f"/kaggle/working/mmlu_med_logs/{model_number}_{subject}.xlsx", index=False)
    
    return EvalResult(subject, n_total, n_correct, acc, eval_time)

def run(subjects: Optional[List[str]] = None, k_shot: int = 5, limit_per_subject: Optional[int] = None, seed: int = 42, sleep: float = 0.0):
    cfg = EvalConfig(subjects=subjects or MEDICAL_SUBJECTS, k_shot=k_shot, limit_per_subject=limit_per_subject, seed=seed, sleep=sleep)
    rng = random.Random(cfg.seed)

    results: List[EvalResult] = []
    for s in cfg.subjects:
        print(f"\n=== Evaluating subject: {s} ===")
        res = evaluate_subject(s, cfg, rng)
        if res is None:
            continue
        print(f"{s}: n={res.n}, acc={res.accuracy:.4f}")
        results.append(res)

    total_n = sum(r.n for r in results)
    total_correct = sum(r.correct for r in results)
    micro_acc = total_correct / max(1, total_n)
    macro_acc = sum(r.accuracy for r in results) / max(1, len(results))

    print("\n===== SUMMARY (medical MMLU) =====")
    for r in results:
        print(f"{r.subject:24s}  acc={r.accuracy:.4f}  n={r.n}")
    print("---------------------------------------------")
    print(f"Micro-avg accuracy: {micro_acc:.4f}")
    # print(f"Macro-avg accuracy: {macro_acc:.4f}")

    import pandas as pd
    df = pd.DataFrame([{ "subject": r.subject, "n": r.n, "correct": r.correct, "accuracy": r.accuracy, "eval_time": r.eval_time } for r in results])
    df.loc[len(df)] = {"subject": "_TOTAL_micro_", "n": total_n, "correct": total_correct, "accuracy": micro_acc}
    df.loc[len(df)] = {"subject": "_MEAN_macro_", "n": len(results), "correct": None, "accuracy": macro_acc}
    df.to_excel(f"/kaggle/working/mmlu_med_summary_{model_number}.xlsx", index=False)
    print(f"\nSaved summary: /kaggle/working/mmlu_med_summary_{model_number}.xlsx")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-21 14:27:10.147367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758464830.494543      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758464830.592260      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# === Настройки модели ===
max_seq_length = 1024
load_in_4bit = True
# model_name = "unsloth/Qwen2-0.5B-bnb-4bit"
model_name = "DimaSK1/Qwen2-0.5B-bnb-4bit-sft_base"
# model_name = 'DimaSK1/Qwen2-1.5B-bnb-4bit_base_sft'
model_number = 'qwen0.5-sft'
# model_name = "unsloth/mistral-7b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.9.7: Fast Bit patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

In [5]:
run(k_shot=5, limit_per_subject=1)


=== Evaluating subject: all ===


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

all/test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

all/validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

...

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Ты увидешь несколько решенных вопросов с правильным ответом.

Вопрос: A 60-year-old man has had painful skin with exfoliation of the skin and mucous membranes for 1 day. He has been taking allopurinol and probenecid for 2 weeks because of gouty arthritis. There is diffuse exfoliation of the skin with oozing of serous fluid. The mucous membranes of the mouth are erythematous and exfoliated. There are no target lesions. Which of the following is the most likely diagnosis?
Варианты ответа:
A) Erythema multiforme
B) Pemphigus erythematosus
C) Staphylococcal scalded-skin syndrome
D) Toxic epidermal necrolysis
Правильный ответ: D

Вопрос: A local library has a scanner to detect library materials that have not been checked out. Each item has a chip somewhere inside. Upon checkout, the chip is deactivated so the scanner will not set off the alarm. The scanner has a 98% chance of detecting an active chip (meaning the material has not been checked out) and setting off the alarm. The scanner also

100%|██████████| 1/1 [00:02<00:00,  2.68s/it]


all: n=1, acc=0.0000

===== SUMMARY (medical MMLU) =====
all                       acc=0.0000  n=1
---------------------------------------------
Micro-avg accuracy: 0.0000

Saved summary: /kaggle/working/mmlu_med_summary_qwen0.5-sft.xlsx


## Визуализация

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# --- Параметры ---
methods = ['tema', 'sft', 'kl_sft']
base_method = 'base'
num_generations = 8
gen_to_plot = 0

# --- Загрузка данных ---
all_data = {}
subjects = set()

# Загружаем данные для базовой модели
base_file_path = Path('mmlu_med_summary_base.xlsx')
if base_file_path.exists():
    df_base = pd.read_excel(base_file_path, engine='openpyxl')
    subj_col = next((c for c in df_base.columns if c.lower() in ["subject", "area", "domain"]), None)
    acc_col = next((c for c in df_base.columns if "acc" in c.lower()), None)

    if subj_col and acc_col:
        df_base = df_base[[subj_col, acc_col]].rename(columns={subj_col: "subject", acc_col: "accuracy"})
        df_base['subject'] = df_base['subject'].replace('_TOTAL_micro_', 'Общая оценка')
        all_data[base_method] = {gen_to_plot: df_base}
        subjects.update(df_base["subject"].unique())
        print("✅ Загружены данные базовой модели")

# Загружаем остальные методы
for method in methods:
    all_data[method] = {}
    file_path = Path(f'mmlu_med_summary_{method}_{gen_to_plot}.xlsx')
    if file_path.exists():
        df = pd.read_excel(file_path, engine='openpyxl')
        subj_col = next((c for c in df.columns if c.lower() in ["subject", "area", "domain"]), None)
        acc_col = next((c for c in df.columns if "acc" in c.lower()), None)
        if subj_col and acc_col:
            df = df[[subj_col, acc_col]].rename(columns={subj_col: "subject", acc_col: "accuracy"})
            df['subject'] = df['subject'].replace('_TOTAL_micro_', 'Общая оценка')
            all_data[method][gen_to_plot] = df
            subjects.update(df["subject"].unique())

subjects = sorted(list(subjects))[1:]

# --- Подготовка данных ---
data_to_plot = {}
for subject in subjects:
    data_to_plot[subject] = []
    for method in [base_method] + methods:
        df = all_data.get(method, {}).get(gen_to_plot)
        if df is not None:
            row = df[df["subject"] == subject]
            data_to_plot[subject].append(row["accuracy"].values[0] if not row.empty else 0)
        else:
            data_to_plot[subject].append(0)

# --- Визуализация ---
plt.style.use("seaborn-v0_8-whitegrid")
all_methods = [base_method] + methods
colors = plt.cm.Paired(np.linspace(0, 1, len(all_methods)))

n_subjects = len(subjects)
n_methods = len(all_methods)
bar_width = 0.8 / n_methods
x = np.arange(n_subjects)

fig, ax = plt.subplots(figsize=(18, 9))

# Рисуем столбцы
for i, method in enumerate(all_methods):
    values = [data_to_plot[sub][i] for sub in subjects]
    offset = (i - n_methods/2 + 0.5) * bar_width
    bars = ax.bar(x + offset, values, bar_width, label=method, color=colors[i], edgecolor="black", alpha=0.85)

    # Подписи значений внутри баров
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f"{height:.2f}",
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3),  # смещение
                    textcoords="offset points",
                    ha="center", va="bottom",
                    fontsize=8)

# Настройка графика
ax.set_xlabel("Датасеты", fontsize=14, labelpad=10)
ax.set_ylabel("Точность", fontsize=14, labelpad=10)
ax.set_title(f"Сравнение моделей на MMLU", fontsize=16, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(subjects, rotation=40, ha="right", fontsize=11)
ax.legend(title="Методы", fontsize=12, title_fontsize=12, frameon=True, shadow=True)
ax.grid(True, axis="y", linestyle="--", alpha=0.6)

plt.tight_layout()
plt.savefig(f"barplot_medical_gen_{gen_to_plot}.png", dpi=300, bbox_inches="tight")
plt.show()

## Оценка модели на бенчмарке MedMCQA

In [1]:
%%capture
!pip install -q --upgrade transformers accelerate peft unsloth

In [2]:
import os
import re
import time
import random
import dataclasses
from dataclasses import dataclass
from typing import List, Dict, Optional, Any
from tqdm import tqdm

import torch
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel

# === Конфигурация ===
USER_TPL = (
    "Вопрос: {question}\n"
    "Варианты ответа:\n"
    "A) {A}\nB) {B}\nC) {C}\nD) {D}\n\n"
    "Ответ:"
)

FEW_SHOT_TPL = (
    "Вопрос: {question}\n"
    "Варианты ответа:\n"
    "A) {A}\nB) {B}\nC) {C}\nD) {D}\n"
    "Правильный ответ: {answer}\n\n"
)

LETTER_TO_INDEX = {"A": 0, "B": 1, "C": 2, "D": 3}
INDEX_TO_LETTER = {v: k for k, v in LETTER_TO_INDEX.items()}


# === Вспомогательные функции ===
def normalize_answer_field(example: Dict[str, Any]) -> Dict[str, Any]:
    q = example["question"]
    choices = [example["opa"], example["opb"], example["opc"], example["opd"]]
    ans = int(example["cop"])  # gold индекс 0..3
    subject = example.get("subject_name", "unknown")
    return {"question": q, "choices": choices, "gold": ans, "subject": subject}


def build_few_shot_block(few_shot_examples: List[Dict[str, Any]]) -> str:
    return "".join(
        FEW_SHOT_TPL.format(
            question=ex["question"], A=ex["choices"][0], B=ex["choices"][1],
            C=ex["choices"][2], D=ex["choices"][3], answer=INDEX_TO_LETTER[ex["gold"]]
        )
        for ex in few_shot_examples
    )


def build_prompt(example: Dict[str, Any], few_shot_block: str = "") -> str:
    A, B, C, D = example["choices"]
    user = USER_TPL.format(question=example["question"], A=A, B=B, C=C, D=D)
    if few_shot_block:
        return ("Ты увидишь несколько решенных вопросов с правильным ответом.\n\n"
                + few_shot_block
                + "Далее ответь на следующий вопрос.\n\n"
                + user)
    return user


def extract_letter(text: str) -> Optional[str]:
    if not text:
        return None
    m = re.search(r"\b([A-Da-d])\b", text.strip())
    if m:
        return m.group(1)
    m = re.search(r"([A-Da-d])\)?\s*$", text.strip())
    return m.group(1) if m else None


# === Генерация с моделью Unsloth ===
def generate(prompt: str, max_new_tokens: int = 128) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):]


@dataclass
class EvalConfig:
    k_shot: int = 5
    limit_per_subject: Optional[int] = 500  # ограничение на примеры
    seed: int = 42
    sleep: float = 0.0


@dataclass
class EvalResult:
    subject: str
    n: int
    correct: int
    accuracy: float


# === Подготовка датасета MedMCQA ===
def load_medmcqa():
    ds = load_dataset("openlifescienceai/medmcqa")
    return ds["validation"], ds["train"]


def pick_few_shots(ds_valid, k: int, rng: random.Random) -> List[Dict[str, Any]]:
    idxs = list(range(len(ds_valid)))
    rng.shuffle(idxs)
    return [normalize_answer_field(ds_valid[i]) for i in idxs[:k]]


def evaluate(cfg: EvalConfig, rng: random.Random):
    valid_split, test_split = load_medmcqa()
    few_shot_block = build_few_shot_block(pick_few_shots(valid_split, cfg.k_shot, rng))

    records = []
    subject_results = []

    # === группируем test_split по subject ===
    df_test = pd.DataFrame([normalize_answer_field(ex) for ex in test_split])
    subjects = df_test["subject"].unique()

    for subj in subjects:
        subj_records = []
        group = df_test[df_test["subject"] == subj]
        lim = min(cfg.limit_per_subject, len(group))
        sampled = group.sample(n=lim, random_state=cfg.seed)

        n_total, n_correct = 0, 0
        print(f'===== SUBJECT: {subj} =====')
        for i in tqdm(range(sampled.shape[0])):
            ex = sampled.iloc[i]
            prompt = build_prompt(ex.to_dict(), few_shot_block)
            raw_out = generate(prompt)
            pred_letter = extract_letter(raw_out) or ""
            pred_idx = LETTER_TO_INDEX.get(pred_letter, -1)
            correct = int(pred_idx == ex["gold"])
            n_total += 1
            n_correct += correct

            records.append({
                "subject": subj,
                "question": ex["question"],
                "A": ex["choices"][0], "B": ex["choices"][1],
                "C": ex["choices"][2], "D": ex["choices"][3],
                "gold_letter": INDEX_TO_LETTER[ex["gold"]],
                "prediction_raw": raw_out.strip(),
                "prediction_letter": pred_letter,
                "correct": correct,
            })
            
            subj_records.append({
                "subject": subj,
                "question": ex["question"],
                "A": ex["choices"][0], "B": ex["choices"][1],
                "C": ex["choices"][2], "D": ex["choices"][3],
                "gold_letter": INDEX_TO_LETTER[ex["gold"]],
                "prediction_raw": raw_out.strip(),
                "prediction_letter": pred_letter,
                "correct": correct,
            })
            
            if cfg.sleep:
                time.sleep(cfg.sleep)

        acc = n_correct / max(1, n_total)
        print(f"{subj}: n = {n_total:.4f}, acc={acc:.4f}")
        subject_results.append(EvalResult(subj, n_total, n_correct, acc))
        pd.DataFrame.from_records(subj_records).to_excel(f"/kaggle/working/medmcqa_logs/{model_number}_{subj}.xlsx", index=False)
    # === общий результат ===
    df = pd.DataFrame.from_records(records)
    n_total = len(df)
    n_correct = df["correct"].sum()
    acc_total = n_correct / max(1, n_total)
    subject_results.append(EvalResult("_TOTAL_", n_total, n_correct, acc_total))

    # === сохранение ===
    # os.makedirs("/kaggle/working/medmcqa_logs", exist_ok=True)
    df.to_excel(f"/kaggle/working/medmcqa_logs/{model_number}_medmcqa.xlsx", index=False)

    df_summary = pd.DataFrame([dataclasses.asdict(r) for r in subject_results])
    df_summary.to_excel(f"/kaggle/working/medmcqa_summary_{model_number}.xlsx", index=False)

    return subject_results


def run(k_shot: int = 5, limit_per_subject: int = 500, seed: int = 42, sleep: float = 0.0):
    cfg = EvalConfig(k_shot=k_shot, limit_per_subject=limit_per_subject, seed=seed, sleep=sleep)
    rng = random.Random(cfg.seed)

    print(f"\n=== Evaluating MedMCQA (with subject_name breakdown) ===")
    os.makedirs("/kaggle/working/medmcqa_logs", exist_ok=True)
    results = evaluate(cfg, rng)

    print("\n===== SUMMARY (MedMCQA) =====")
    for r in results:
        print(f"{r.subject:30s}  acc={r.accuracy:.4f}  n={r.n}")
    return results


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-21 13:26:49.951600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755782810.327034      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755782810.432844      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# === Настройки модели ===
max_seq_length = 1024
load_in_4bit = True
# model_name = "unsloth/Qwen2-1.5B-bnb-4bit"
model_name = "DimaSK1/Qwen2-1.5B-bnb-4bit_new_trainer"
model_number = 'kl_sft_0'
# model_name = "unsloth/mistral-7b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.0M [00:00<?, ?B/s]

Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
run(k_shot=5, limit_per_subject=100)


=== Evaluating MedMCQA (with subject_name breakdown) ===


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

...

100%|██████████| 100/100 [00:20<00:00,  4.77it/s]


Skin: n = 100.0000, acc=0.4300
===== SUBJECT: Unknown =====


100%|██████████| 100/100 [00:22<00:00,  4.54it/s]


Unknown: n = 100.0000, acc=0.3900

===== SUMMARY (MedMCQA) =====
Anatomy                         acc=0.3600  n=100
Biochemistry                    acc=0.4600  n=100
Surgery                         acc=0.4900  n=100
Ophthalmology                   acc=0.3800  n=100
Physiology                      acc=0.4400  n=100
Social & Preventive Medicine    acc=0.4700  n=100
Gynaecology & Obstetrics        acc=0.4200  n=100
Anaesthesia                     acc=0.4100  n=100
Psychiatry                      acc=0.5000  n=100
Microbiology                    acc=0.4200  n=100
Medicine                        acc=0.4400  n=100
Pharmacology                    acc=0.4400  n=100
Dental                          acc=0.3200  n=100
ENT                             acc=0.3800  n=100
Forensic Medicine               acc=0.3800  n=100
Pediatrics                      acc=0.4100  n=100
Orthopaedics                    acc=0.3500  n=100
Radiology                       acc=0.3300  n=100
Pathology                       acc

[EvalResult(subject='Anatomy', n=100, correct=36, accuracy=0.36),
 EvalResult(subject='Biochemistry', n=100, correct=46, accuracy=0.46),
 EvalResult(subject='Surgery', n=100, correct=49, accuracy=0.49),
 EvalResult(subject='Ophthalmology', n=100, correct=38, accuracy=0.38),
 EvalResult(subject='Physiology', n=100, correct=44, accuracy=0.44),
 EvalResult(subject='Social & Preventive Medicine', n=100, correct=47, accuracy=0.47),
 EvalResult(subject='Gynaecology & Obstetrics', n=100, correct=42, accuracy=0.42),
 EvalResult(subject='Anaesthesia', n=100, correct=41, accuracy=0.41),
 EvalResult(subject='Psychiatry', n=100, correct=50, accuracy=0.5),
 EvalResult(subject='Microbiology', n=100, correct=42, accuracy=0.42),
 EvalResult(subject='Medicine', n=100, correct=44, accuracy=0.44),
 EvalResult(subject='Pharmacology', n=100, correct=44, accuracy=0.44),
 EvalResult(subject='Dental', n=100, correct=32, accuracy=0.32),
 EvalResult(subject='ENT', n=100, correct=38, accuracy=0.38),
 EvalResult(